## Road-snapping workflow on Caspar data

In [5]:
# Format Caspar's data for same workflow as all_plt_data
# import pandas as pd

# all_caspar_gps = pd.read_excel('data/caspar_data.xlsx')
# all_caspar_gps['cst_datetime'] = pd.to_datetime(all_caspar_gps['timestamp'])

# # Reformat colnames
# all_caspar_gps.drop(columns=['timestamp'], inplace=True)
# all_caspar_gps.rename(columns={'latitude': 'lat', 'longitude': 'long'}, inplace=True)

# all_caspar_gps.to_csv('data/caspar_data.csv', index=False)

In [27]:
import pandas as pd

all_caspar_gps = pd.read_csv('data/caspar_data.csv')

In [28]:
def filter_date(all_caspar_gps, date):
    """Filter for a specific date in the format 'YYYY-MM-DD'"""
    return all_caspar_gps[all_caspar_gps['cst_datetime'].dt.date == pd.to_datetime(date).date()]

# Show all available different dates for caspar's data
def get_all_dates(all_caspar_gps):
    """Return all unique dates in the Caspar data"""
    all_caspar_gps['cst_datetime'] = pd.to_datetime(all_caspar_gps['cst_datetime'])
    all_caspar_gps['date'] = all_caspar_gps['cst_datetime'].dt.strftime('%Y-%m-%d')

    caspar_dates = all_caspar_gps.groupby('date').size().reset_index(name='gps_counts')
    return caspar_dates

caspar_dates = get_all_dates(all_caspar_gps)

In [24]:
caspar_data = filter_date(all_caspar_gps, '2014-09-10')
caspar_data

,lat,long,cst_datetime,date
938,40.702033,-73.983259,2014-09-10 00:00:58.476,2014-09-10
939,40.702039,-73.983376,2014-09-10 00:01:58.482,2014-09-10
940,40.701983,-73.983205,2014-09-10 00:02:58.459,2014-09-10
941,40.701908,-73.983472,2014-09-10 00:03:58.494,2014-09-10
942,40.702075,-73.983924,2014-09-10 00:05:03.507,2014-09-10
...,...,...,...,...
1875,40.702027,-73.983165,2014-09-10 23:55:09.462,2014-09-10
1876,40.702016,-73.983215,2014-09-10 23:56:14.468,2014-09-10
1877,40.701978,-73.983311,2014-09-10 23:57:14.451,2014-09-10
1878,40.701787,-73.983417,2014-09-10 23:58:14.522,2014-09-10


In [16]:
from scripts.TimeSegmentation import Segment
from scripts.RoadSnapping import RoadSnap
from scripts.PlotMap import PlotMap

In [25]:
# Segment + kalman filter ('ksegment')
caspar_segment = Segment.segment_df(caspar_data, 300)
caspar_ksegment = Segment.kalman_filter_segments(caspar_segment)

# Roadsnapping
caspar_snapjson = RoadSnap.snap_roads(caspar_segment)

caspar_snapdf = RoadSnap.make_snapdf(caspar_snapjson)
caspar_legdf = RoadSnap.make_legdf(caspar_snapjson)
caspar_tracedf = RoadSnap.make_tracedf(caspar_snapjson)

Split indices: [0, 260, 279, 324, 334, 335, 336, 337, 361, 373, 394, 438, 467, 502, 514, 553, 572, 573, 574, 633, 699, 703, 704, 705, 707, 809, 855, 941]
note: df has <2 rows, skipping kalman filter
note: df has <2 rows, skipping kalman filter
note: df has <2 rows, skipping kalman filter
note: df has <2 rows, skipping kalman filter
note: df has <2 rows, skipping kalman filter
note: df has <2 rows, skipping kalman filter
note: df has <2 rows, skipping kalman filter


In [26]:
pm = PlotMap(caspar_segment)
pm.full_polyline(caspar_segment, 'original')
pm.full_polyline(caspar_ksegment, 'kalman')

pm.segment_polyline(caspar_ksegment, 'kalman')

pm.circles(caspar_segment, 'original')
pm.circles(caspar_ksegment, 'kalman')
pm.show()